
https://www.kaggle.com/datatattle/covid-19-nlp-text-classification

Columns:

1) Location

2) Tweet At

3) Original Tweet

4) Label

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import normalize # normalization of the matrix
import scipy
import pandas as pd
import unicodedata, re, string
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
import itertools

In [ ]:
train_df = pd.read_csv(r'/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv', encoding='latin_1')
test_df = pd.read_csv(r'/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv', encoding='latin_1')

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

In [ ]:
class Mish(keras.layers.Activation):
    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'
def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))
keras.utils.get_custom_objects().update({'mish': Mish(mish)})

In [ ]:
train_df.head()

In [ ]:
test_df.head()

Pro řešení problemu jsem se rozhodl využít pouze text a proto Dále již nepracuji s datem lokací nebo se jmény.
Dále jsem si upravil třidy kde jsem původních 5 třid rozdělil na 2 pozitivní a negativní s tím že neutrálni jsem zařadil do skupiny s pozitivními

In [ ]:
train_df = train_df.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
test_df = test_df.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
train_df["Sentiment"].replace({"Neutral": 1, "Positive": 1, "Extremely Negative": 0, "Negative": 0, "Extremely Positive": 1}, inplace=True)
test_df["Sentiment"].replace({"Neutral": 1, "Positive": 1, "Extremely Negative": 0, "Negative": 0, "Extremely Positive": 1}, inplace=True)

In [ ]:
train_df.shape

In [ ]:
sns.countplot(x='Sentiment', data=train_df)

Počet pozitivních a počet a negativních zpráv

In [ ]:
train_df.Sentiment.value_counts()

In [ ]:
train_df['length'] = train_df.OriginalTweet.apply(len)

In [ ]:
sns.barplot(x='Sentiment', y='length', data = train_df)

Ukazka originálních zpráv

In [ ]:
for x in train_df.loc[:10, 'OriginalTweet']:
    print(x)
    print('---------')


Metody a funkce pro upravení textu v tweetu

In [ ]:
def remove_users_http(word):
    """Remove non-ASCII characters from list of tokenized words"""
    word = re.sub(r'http\S+', '', word)
    word = re.sub(r'@\w+', '', word)
    word = re.sub(r'#\w+', '', word)
    return word

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_numbers(words):
    """Remove all interger occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        new_word = re.sub("\d+", "", word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_numbers(words)
    words = remove_stopwords(words)
    return words

def form_sentence(tweet):
    tweet = remove_users_http(tweet)
    tweet_blob = TextBlob(tweet)
    return tweet_blob.words

In [ ]:
train_df['Clean_text'] = train_df['OriginalTweet'].apply(form_sentence)
test_df['Clean_text'] = test_df['OriginalTweet'].apply(form_sentence)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df['Clean_text'] = train_df['Clean_text'].apply(normalize)
test_df['Clean_text'] = test_df['Clean_text'].apply(normalize)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
def fix_nt(words):
    st_res = []
    for i in range(0, len(words) - 1):
        if words[i+1] == "n't" or words[i+1] == "nt":
            st_res.append(words[i]+("n't"))
        else:
            if words[i] != "n't" and words[i] != "nt":
                st_res.append(words[i])
    return st_res

In [ ]:
train_df['Clean_text'] = train_df['Clean_text'].apply(fix_nt)
test_df['Clean_text'] = test_df['Clean_text'].apply(fix_nt)

In [ ]:
train_df['Clean_text'] = train_df['Clean_text'].apply(lambda x: " ".join(x))
test_df['Clean_text'] = test_df['Clean_text'].apply(lambda x: " ".join(x))

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
all_words = list(itertools.chain(*train_df.Clean_text))

In [ ]:
dist = nltk.FreqDist(all_words)
dist

In [ ]:
len(dist)

In [ ]:
max(train_df.Clean_text.apply(len))

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
embedding_dim = 128 # Dimension of embedded representation - this is already part of latent space, there is captured some dependecy among words, we are learning this vectors in ANN
vocab_size = 10000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing - words in vectorited representation are independent

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(train_df.Clean_text.values)

Rozdělení trenovací sady sady na trenovaci a validační

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_df.Clean_text, train_df.Sentiment, test_size=0.1, random_state=13)

připravení testovací sady

In [ ]:
X_test = test_df.Clean_text
y_test = test_df.Sentiment

In [ ]:
print('Vocabulary example: ', vect_layer.get_vocabulary()[:10])
print('Vocabulary shape: ', len(vect_layer.get_vocabulary()))

In [ ]:
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='mish', return_sequences=True)(emb)
x = GRU(64, activation='mish', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, 'mish')(x)
x = keras.layers.Dense(64, 'mish')(x)
x = keras.layers.Dense(32, 'mish')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()

model.compile(optimizer='rmsprop', loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])


In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 128
epochs = 7
history = model.fit(X_train.values, y_train.values, validation_data=(X_valid.values, y_valid.values), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
show_history(history)

In [ ]:
y_test_loss, accuracy = model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test).ravel()

In [ ]:
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_pred)

In [ ]:
f1_score(y_true=y_test, y_pred=y_pred)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))

In [ ]:
print(confusion_matrix(y_true=y_test, y_pred=y_pred))